In [12]:
import astropy
import astropy.units as u
import numpy as np
import specutils
import matplotlib.pyplot as plt
import pandas as pd
import inspect
import FunctionLib as FL

import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, append=True)

In [13]:
DJA_File_Path_str=os.path.expanduser('~/DJAData/')
DJA_Catalog_DataFrame=pd.read_csv('./DJACatalog.csv')

if os.path.exists(DJA_File_Path_str):
    DJA_File_list=np.array(os.listdir(DJA_File_Path_str))
print('Number of files in DJA directory:', len(DJA_File_list))

Number of files in DJA directory: 18506


In [17]:
Sample_Indices_list=list()
for Index_In_DJA_Catalog_int in range(len(DJA_Catalog_DataFrame)):

    Redshift_float=DJA_Catalog_DataFrame['z'][Index_In_DJA_Catalog_int]

    if (Redshift_float<4)or (Redshift_float>6.5):
        continue

    Spectra_NGrating_int=DJA_Catalog_DataFrame.nGr[Index_In_DJA_Catalog_int]
    if Spectra_NGrating_int<2:
        continue

    Sample_Indices_list.append(Index_In_DJA_Catalog_int)

print('Sample number:', len(Sample_Indices_list))
Sample_Indices_list=np.array(Sample_Indices_list)

FL.Free('Index_In_DJA_Catalog_int')

Sample number: 620


In [ ]:
for DJA_Catalog_DataFrame_Index_int in Sample_Indices_list:

    Object_Catalog_Dataframe=DJA_Catalog_DataFrame[DJA_Catalog_DataFrame.index==DJA_Catalog_DataFrame_Index_int]

    Object_Prism_fits_File_Name_str=Object_Catalog_Dataframe.file.values[0]
    Object_NGrating_int=Object_Catalog_Dataframe.nGr.values[0]
    Object_Grating_List_str=Object_Catalog_Dataframe.Grating.values[0]

    for Grating_Type_str in Object_Grating_List_str.split('   '):
        Grating_Type_str=Grating_Type_str.strip().lower()

        Disperser_Name_str=Grating_Type_str[1:-1]

        Dual_Filter_Flag=False

        if Disperser_Name_str=='140':
            Dual_Filter_Flag=True
            Disperser_Filter_Combination_070_str=Grating_Type_str+'-f070lp'
            Disperser_Filter_Combination_100_str=Grating_Type_str+'-f100lp'
        elif Disperser_Name_str=='235':
            Dual_Filter_Flag=False
            Disperser_Filter_Combination_170_str=Grating_Type_str+'-f170lp'
        elif Disperser_Name_str=='395':
            Dual_Filter_Flag=False
            Disperser_Filter_Combination_290_str=Grating_Type_str+'-f290lp'

        if Dual_Filter_Flag:





    break

Grating Type: g140m
Grating Type: g235m
Grating Type: g395m


'G140M   G235M   G395M'